<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Two: Exploratory data analysis and feature extraction. </h2>	


<h5> In this section, Compute at least 15 such image features (a method for each), including the following (NOTE: At least 10 of these must be scalar features and 2 matrix-based features): (i) image size, (ii) average of the red-channel intensity, (iii) aspectratio. This will require significant explatoratory research and data analysis. The first one is already implemented for you, and the next two are pre-specified. Additional requirements specfied in pdf. </h5>

In [1]:
def ft0(image):
    return image.size

def ft1(image):
    raise notImplementedError()

def ft2(image):
    raise notImplementedError()

def ft3(image):
    raise notImplementedError()

def ft4(image):
    raise notImplementedError()

def ft5(image):
    raise notImplementedError()

def ft6(image):
    raise notImplementedError()

def ft7(image):
    raise notImplementedError()

def ft8(image):
    raise notImplementedError()

def ft9(image):
    raise notImplementedError()

def ft9(image):
    raise notImplementedError()
    
def ft10(image):
    raise notImplementedError()
    
def ft11(image):
    raise notImplementedError()
    
def ft12(image):
    raise notImplementedError()
    
def ft13(image):
    raise notImplementedError()
    
def ft14(image):
    raise notImplementedError()
    
def ft15(image):
    raise notImplementedError()
    
    
#Returns the aspect ratio of the image


 We expect all external sources sited, and significant indication of EDA. 

<h4> Graphs </h4>

<h4> Sources </h4>

<h4> DataFrame Creation </h4>

In [2]:
def feature_frame(df):
    return df
    #Returns data-frame with all the features now inside, and calculated


In [3]:
feature_frame(data_from_nb1)

NameError: name 'starting_data' is not defined